# IPython EUPS magics

### What

This IPython extension adds a new `%eups` magic that allows one to call EUPS from IPython, setup products into the current IPython notebook's environment, and setup EUPS itself.

This comes in very handy when we want the Python notebook to fully capture the products (possibly down to the product version) that were used to execute it.

It's also handy when in the middle of some analysis you realize you've forgotten to setup a product you need.

### What: tl;dr version

In [1]:
%load_ext eups_magic

%eups list
%eups setup oorb lsst-g650e0a6f6c

import pyoorb
pyoorb.__file__

Initializing EUPS from /Users/mjuric/.eups/default
Using EUPS_PATH: /Users/mjuric/projects/eups/stack
oorb                  lsst-g650e0a6f6c


'/Users/mjuric/projects/eups/stack/DarwinX86/oorb/lsst-g650e0a6f6c/python/pyoorb.so'

## Installing and running

### From a packed script

This is by far the quickest way to get started. Just download the `ipython` pass-through script and place it somewhere on your path (before the real `ipython` !):

    wget http://eupsforge.net/ipython
    cp ipython ...somewhere/on/your/path...

This will ensure ipython is always started with EUPS magics support.

### From EUPS package

This extension is also available as an EUPS package at EUPSForge:

    eups distrib install -r http://eupsforge.net ipython_eups
    setup ipython_eups

Once you've installed it, you may want to automatically load it every time you set up your EUPS environment. E.g., if you're using `loadLSST.bash` (or any of the `loadLSST.*` scripts), open it in an editor and append:

    setup ipython_eups

to the end.

### From source

The sources are on github at http://github.com/mjuric/ipython_eups. Running from source is pretty straightforward as well:

    git clone http://github.com/mjuric/ipython_eups
    setup -r ipython_eups

This is a pure python extenson, so no compilation is necessary.

## Initializing EUPS

To activate the `%eups` magic, load the `eups_magic` extension. It's best to begin your notebooks with this: 

In [1]:
%load_ext eups_magic

The message above tells us we've forgotten to setup EUPS itself (i.e., by sourcing setups.sh script). That's fine; we can do it from the notebook itself:

In [2]:
%eups init /Users/mjuric/projects/eups

Initializing EUPS from /Users/mjuric/projects/eups
Using EUPS_PATH: /Users/mjuric/projects/eups/stack
To set this EUPS as default, rerun as `%eups init --set-default /Users/mjuric/projects/eups'


We can also tell `%eups` to remember the path to default EUPS: 

In [3]:
%eups init --set-default /Users/mjuric/projects/eups

Initializing EUPS from /Users/mjuric/projects/eups
Using EUPS_PATH: /Users/mjuric/projects/eups/stack
Setting as default: /Users/mjuric/projects/eups symlinked to /Users/mjuric/.eups/default


... and now we can run `%eups init` without an argument:

In [4]:
%eups init

Initializing EUPS from /Users/mjuric/.eups/default
Using EUPS_PATH: /Users/mjuric/projects/eups/stack


Also, now that we have the default set up, `%eups init` will automatically run when the extension is loaded so nothing is needed beyond `%load_ext eups_magic`.

## Basic capabilities

The `%eups` magic lets you run any EUPS subcommand. For example, let's see which products are available:

In [5]:
%eups list

oorb                  lsst-g650e0a6f6c


Let's try to import a module from a package that hasn't been `setup`-ed yet:

In [6]:
import pyoorb

ImportError: No module named pyoorb

OK, so we'll set it up first:

In [7]:
%eups setup oorb lsst-g650e0a6f6c

Let's check if this worked:

In [8]:
%eups list -s

oorb                  lsst-g650e0a6f6c 	setup


... and now we can use its modules directly from this notebook:

In [9]:
import pyoorb

We can also unsetup a module.

***Hint***: *Note however that because of how Python handles modules, this will not unload an already loaded module. Furthermore, while pure-Python modules can be reloaded with `reload()`, that is not possible for C extensions (see http://bugs.python.org/issue1144263).*

Bottom line: this will not enable you to switch out a version of a module at runtime, unless the module is pure Python and you remember to call reload.

In [10]:
%eups unsetup oorb lsst-g650e0a6f6c
%eups list -s

eups list: No products found


## Miscellaneous

Errors get appropriately highlighted:

In [11]:
%eups foobar

ERROR: Command 'eups foobar ' returned non-zero exit status 10
eups foobar: Unrecognized command: foobar



We can setting up a list of products in a loop, using IPython variable expansion:

In [12]:
products = [('oorb', 'lsst-g650e0a6f6c')]
for product, version in products:
    %eups setup $product $version
%eups list -s

oorb                  lsst-g650e0a6f6c 	setup


All EUPS subcommands work:

In [13]:
%eups distrib list -s http://eupsforge.net

  anaconda             generic    2.1.0-lsst1
  anaconda             generic    2.1.0-4-g35ca374
  ann                  generic    master-g241439a636
  eupsforge            generic    master-g9ae1b6a1f0
  eupsforge_distrib    generic    master-g0740fbfb19
  eupsforge_distrib    generic    master-g0740fbfb19+3
  eupsforge_distrib    generic    master-g0740fbfb19+4
  eupsforge_distrib    generic    master-g4bfbf8a310
  eupsforge_distrib    generic    master-g5126290247+1
  eupsforge_distrib    generic    master-g5126290247+3
  eupsforge_distrib    generic    master-g67d36cbb01
  eupsforge_distrib    generic    master-g704909cfe0
  eupsforge_distrib    generic    master-gb7d5b0be39
  firefly              generic    master-g54a0dce6ee
  firefly              generic    tickets.DM-2731-eups-for-firefly-g9c17782531
  firefly              generic    tickets.DM-2731-eups-for-firefly-geffca691b1
  git                  generic    1.8.5.2
  git                  generic    2.4.0
  git              

## Internals

In [14]:
import os
linkfarm = os.environ['IPYTHON_EUPS_LIB_LINKFARM']
print os.environ['DYLD_LIBRARY_PATH']
print linkfarm
! ls -l $linkfarm
! ls -l $linkfarm/lib/

/Users/mjuric/projects/eups/stack/DarwinX86/oorb/lsst-g650e0a6f6c/lib:/var/folders/fn/xlbw59n15rg8ywvrxdy9xfm00000gn/T/ipython-eups-linkfarm.XXXXX.HliWLSPS/lib
/var/folders/fn/xlbw59n15rg8ywvrxdy9xfm00000gn/T/ipython-eups-linkfarm.XXXXX.HliWLSPS
total 8
drwx------  4 mjuric  staff  136 Jun 20 02:15 ipython-eups-linkfarm.17250.ClI3YJ
lrwxr-xr-x  1 mjuric  staff   34 Jun 20 02:15 lib -> ipython-eups-linkfarm.17250.ClI3YJ
drwxr-xr-x  4 mjuric  staff  136 Jun 20 02:12 python
total 16
lrwxr-xr-x  1 mjuric  staff  78 Jun 20 02:15 Makefile -> /Users/mjuric/projects/eups/stack/DarwinX86/oorb/lsst-g650e0a6f6c/lib/Makefile
lrwxr-xr-x  1 mjuric  staff  83 Jun 20 02:15 liboorb.dylib -> /Users/mjuric/projects/eups/stack/DarwinX86/oorb/lsst-g650e0a6f6c/lib/liboorb.dylib
